In [2]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

# test()

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle


def load_cifar_batch(file):
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    return batch

def load_cifar_batches(files):
    all_images = []
    all_labels = []
    for file in files:
        with open(file, 'rb') as fo:
            batch = pickle.load(fo, encoding='bytes')
        all_images.append(batch[b'data'].reshape((-1, 3, 32, 32)).astype(np.float32) / 255.0)
        all_labels.append(batch[b'labels'])
    return np.concatenate(all_images), np.concatenate(all_labels)


train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), # Randomly flip the images on the horizontal axis
    transforms.RandomRotation(10), # Randomly rotate the images by +/- 10 degrees
    transforms.RandomCrop(32, padding=4), # Apply random crops
    transforms.ToTensor(), # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Normalize images
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# batch_files = ['cifar-10-batches-py/data_batch_1', 'cifar-10-batches-py/data_batch_2', 'cifar-10-batches-py/data_batch_3', 'cifar-10-batches-py/data_batch_4', 'cifar-10-batches-py/data_batch_5']
# train_images_dict, train_labels = load_cifar_batches(batch_files)

# # print(len(train_images_dict))
# train_images = train_images_dict.reshape((-1, 3, 32, 32)).astype(np.float32) / 255.0 
# # train_labels = train_labels_dict[b'labels']

# train_images_tensor = torch.tensor(train_images, dtype=torch.float32)
# train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)

# trainDataLoader = torch.utils.data.DataLoader(train_images_tensor, batch_size=64,shuffle=True)
# trainDataLoaderLabels = torch.utils.data.DataLoader(train_labels_tensor, batch_size=64,shuffle=True)

# ~~~~~~~~~~~~~~~~

# batch_1_dict = load_cifar_batch('cifar-10-batches-py/data_batch_3')
# train_images = batch_1_dict[b'data'].reshape((-1, 3, 32, 32)).astype(np.float32) / 255.0 
# train_labels = batch_1_dict[b'labels']



# ~~~~~~~~~~~~~~~~

batch_files = ['cifar-10-batches-py/data_batch_1', 'cifar-10-batches-py/data_batch_2', 'cifar-10-batches-py/data_batch_3', 'cifar-10-batches-py/data_batch_4', 'cifar-10-batches-py/data_batch_5']
trainarr = None
trainlabel = None

for file in batch_files:
    # Load CIFAR-10 batch
    batch_data = load_cifar_batch(file)
    
    # Extract images and labels
    images = batch_data[b'data']  # Assuming 'data' contains the images
    labels = batch_data[b'labels']  # Assuming 'labels' contains the corresponding labels
    
    # Reshape images to (-1, 3, 32, 32) and normalize
    images = images.reshape((-1, 3, 32, 32)).astype(np.float32) / 255.0
    
    # Append images and labels to trainarr and trainlabel respectively
    if trainarr is None:
        trainarr = images
        trainlabel = labels
    else:
        trainarr = np.concatenate((trainarr, images), axis=0)
        trainlabel = np.concatenate((trainlabel, labels), axis=0)


train_images_tensor = torch.tensor(trainarr, dtype=torch.float32)
train_labels_tensor = torch.tensor(trainlabel, dtype=torch.long)


trainDataLoader = torch.utils.data.DataLoader(train_images_tensor, batch_size=64,shuffle=True)
trainDataLoaderLabels = torch.utils.data.DataLoader(train_labels_tensor, batch_size=64,shuffle=True)

# ~~~~~~~~~~~~~~~~



testdata = load_cifar_batch('cifar-10-batches-py/test_batch')
test_images = testdata[b'data'].reshape((-1, 3, 32, 32)).astype(np.float32) / 255.0 
test_labels = testdata[b'labels']

test_images_tensor = torch.tensor(test_images, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)


testDataLoader = torch.utils.data.DataLoader(test_images_tensor, batch_size=64,shuffle=True)
testDataLoaderLabels = torch.utils.data.DataLoader(test_labels_tensor, batch_size=64,shuffle=True)


images = next(iter(trainDataLoader))
labels = next(iter(trainDataLoaderLabels))
print(len(trainDataLoader))


782


In [ ]:
model = ResNet18().cuda()
loss = torch.nn.CrossEntropyLoss() # Step 2: loss
# optimizer = torch.optim.SGD(model.parameters(), lr=.01)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

train_loss_history = []
test_loss_history = []
matches = 0
total = 0

# for epoch in range(3):
#   train_loss = 0.0
#   test_loss = 0.0

#   model.train()
#   for i, data in enumerate(zip(trainDataLoader, trainDataLoaderLabels)):
#     images, labels = data
#     images = images.cuda()
#     labels = labels.cuda()
#     optimizer.zero_grad() # zero out any gradient values from the previous iteration
#     predicted_output = model(images) # forward propagation
#     fit = loss(predicted_output, labels)  # calculate our measure of goodness
#     fit.backward() # backpropagation
#     optimizer.step() # update the weights of our trainable parameters
#     train_loss += fit.item()
#   train_loss = train_loss / len(trainDataLoader)
#   train_loss_history += [train_loss]
#   print(f'Epoch {epoch}, Train loss {train_loss}')


for epoch in range(20):
  train_loss = 0.0
  test_loss = 0.0

  model.train()
  for i, data in enumerate(zip(trainDataLoader, trainDataLoaderLabels)):
    images, labels = data
    images = images.cuda()
    labels = labels.cuda()
    optimizer.zero_grad() # zero out any gradient values from the previous iteration
    predicted_output = model(images) # forward propagation
    fit = loss(predicted_output, labels)  # calculate our measure of goodness
    fit.backward() # backpropagation
    optimizer.step() # update the weights of our trainable parameters
    train_loss += fit.item()

  model.eval()
  for i, data in  enumerate(zip(testDataLoader, testDataLoaderLabels)):
    with torch.no_grad():
      images, labels = data
      images = images.cuda()
      labels = labels.cuda()
      predicted_output = model(images)
      fit = loss(predicted_output, labels)
      test_loss += fit.item()
      _, predicted = torch.max(predicted_output.data, 1)
      matches += (predicted == labels).sum().item()
      total += labels.size(0)
  train_loss = train_loss / len(trainDataLoader)
  test_loss = test_loss / len(testDataLoader)
  train_loss_history += [train_loss]
  test_loss_history += [test_loss]
  print(f'Epoch {epoch}, Train loss {train_loss}, Test loss {test_loss}')

Epoch 0, Train loss 2.377144864148191, Test loss 2.307589204448044
Epoch 1, Train loss 2.306876627990352, Test loss 2.3068732304178226
Epoch 2, Train loss 2.3063747904184835, Test loss 2.304910532228506
Epoch 3, Train loss 2.306922726009203, Test loss 2.3052359614402627
Epoch 4, Train loss 2.3058435858972848, Test loss 2.308054509436249
Epoch 5, Train loss 2.3064606762907998, Test loss 2.3054943707338564


In [12]:
correct = 0
total = 0
model.eval()
for i, data in  enumerate(zip(testDataLoader, testDataLoaderLabels)):
  images, labels = data
  images = images.cuda()
  labels = labels.cuda()
  with torch.no_grad():
    predicted_output = model(images)
    _, predicted = torch.max(predicted_output.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Accuracy of the network on the test images: %d %%' % (100 * accuracy))

Accuracy of the network on the test images: 9 %
